In [72]:
import cassiopeia as cass
from riot_api import get_puuid
from API_KEY import API_KEY
cass.set_riot_api_key(API_KEY)
api_key = API_KEY

puuid = get_puuid('kitkat','miffy','americas', api_key=api_key)
a_summoner = cass.get_summoner(puuid=puuid,region="NA")

In [73]:
match = a_summoner.match_history[19]
from cassiopeia import Lane

for participant in match.participants:
    if(participant.lane == Lane.top_lane):
        print("hi")
    if participant.summoner == a_summoner:
        print(participant.lane)
        print(participant.champion.name)
        for i, event in enumerate(participant.timeline.events):
            print(f"Type: {event.type}, time: {event.timestamp}")
            if event.type in {"CHAMPION_KILL", "ELITE_MONSTER_KILL", "BUILDING_KILL"}:
                print(event.position)


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/cmDcscqjKchTj0kJnY176BcBRtLWANM_ZGf_GHmZuD5cEe3lb-MukzvrfDc7GJmTb4IRGSW6Xh6kUA/ids?start=0&count=100
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5091058666
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/cmDcscqjKchTj0kJnY176BcBRtLWANM_ZGf_GHmZuD5cEe3lb-MukzvrfDc7GJmTb4IRGSW6Xh6kUA
None
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/14.16.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Seraphine
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_5091058666/timeline
Type: ITEM_PURCHASED, time: 0:00:36.363000
Type: SKILL_LEVEL_UP, time: 0:00:39.369000
Type: SKILL_LEVEL_UP, time: 0:00:39.870000
Type: SKILL_LEVEL_UP, time: 0:00:40.271000
Ty

In [36]:
#match = a_summoner.match_history[19]
match_list = [
    7119039473,
    7118715104,
    7118663922,
    7117777221,
    7117725931,
    7117717382,
    7116157145,
    7115572750,
    7115540086,
    7115466894,
    7113589500,
    7113555422,
    7113463351,
    7113445822,
    7113375986
]
puuid = get_puuid('아따머스마야','kr1','asia', api_key=api_key)
a_summoner = cass.get_summoner(puuid=puuid, region="KR")

for match_id in match_list:
    match = cass.get_match(id=match_id,region="KR")
    timeline = match.timeline
    for participant in match.participants:
        if participant.summoner == a_summoner:
            id = participant.id
            summoner_lane = participant.lane
            summoner_team = participant.team

    opponent_id = None
    for participant in match.participants:
        if participant.lane == summoner_lane and participant.team != summoner_team:
            opponent_id = participant.id

    for minute, frame in enumerate(timeline.frames, start=0):
        participant_frame = frame.participant_frames[id]
        opponent_frame = frame.participant_frames[opponent_id]
        #print(f"CS: {participant_frame.creep_score - opponent_frame.creep_score}, Gold: {participant_frame.gold_earned - opponent_frame.gold_earned}, XP: {participant_frame.experience - opponent_frame.experience}, Score: {participant_frame.position}")

        
        

https://asia.api.riotgames.com/riot/account/v1/accounts/by-riot-id/아따머스마야/kr1?api_key=RGAPI-28887941-ef83-4b09-869e-a51fd2e2b671
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False


In [48]:
import pandas as pd

# Initialize lists to store the data
cs_diffs = []
gold_diffs = []
xp_diffs = []
times = []
#win = []

match_list = [
    7075054854,
    7072826124,
    7071975428,
    7071690356,
    7061656764,
    7061304802,
    7060386461,
    7052654476,
    7048033517,
    7047818263,
    7047460259,
    7047193616,
    7046676172,
    7042849501,
    7037464442,
    7030276514,
    7024816018,
    7024448687
    ]
puuid = get_puuid('adsmurf','true','europe', api_key=api_key)
a_summoner = cass.get_summoner(puuid=puuid, region="EUW")

data_records = []

# Loop through each match
for match_id in match_list:
    match = cass.get_match(id=match_id, region="EUW")
    timeline = match.timeline
    
    # Find the participant ID and lane for the summoner
    for participant in match.participants:
        if participant.summoner == a_summoner:
            id = participant.id
            summoner_lane = participant.lane
            summoner_team = participant.team
            win = participant.stats.win
    
    # Find the opponent in the same lane but different team
    opponent_id = None
    for participant in match.participants:
        if participant.lane == summoner_lane and participant.team != summoner_team:
            opponent_id = participant.id
    
    # Initialize variables to store differences
    cs_diff_5, cs_diff_10, cs_diff_15 = None, None, None
    gold_diff_5, gold_diff_10, gold_diff_15 = None, None, None
    xp_diff_5, xp_diff_10, xp_diff_15 = None, None, None

    # Loop through each frame in the timeline
    for minute, frame in enumerate(timeline.frames, start=1):
        participant_frame = frame.participant_frames[id]
        opponent_frame = frame.participant_frames[opponent_id]
        
        # Calculate differences at 14, 21, and 30 minutes
        if minute == 5:
            cs_diff_5 = participant_frame.creep_score - opponent_frame.creep_score
            gold_diff_5 = participant_frame.gold_earned - opponent_frame.gold_earned
            xp_diff_5 = participant_frame.experience - opponent_frame.experience
        elif minute == 10:
            cs_diff_10 = participant_frame.creep_score - opponent_frame.creep_score
            gold_diff_10 = participant_frame.gold_earned - opponent_frame.gold_earned
            xp_diff_10 = participant_frame.experience - opponent_frame.experience
        elif minute == 15:
            cs_diff_15 = participant_frame.creep_score - opponent_frame.creep_score
            gold_diff_15 = participant_frame.gold_earned - opponent_frame.gold_earned
            xp_diff_15 = participant_frame.experience - opponent_frame.experience

    # Append the data to the records list
    data_records.append({
        'match_id': match_id,
        'cs_diff_5': cs_diff_5,
        'cs_diff_10': cs_diff_10,
        'cs_diff_15': cs_diff_15,
        'gold_diff_5': gold_diff_5,
        'gold_diff_10': gold_diff_10,
        'gold_diff_15': gold_diff_15,
        'xp_diff_5': xp_diff_5,
        'xp_diff_10': xp_diff_10,
        'xp_diff_15': xp_diff_15,
        'win': win
    })

df = pd.DataFrame(data_records)

print(df)



https://europe.api.riotgames.com/riot/account/v1/accounts/by-riot-id/adsmurf/true?api_key=RGAPI-28887941-ef83-4b09-869e-a51fd2e2b671
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7075054854
Making call: https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/uE8Tw1WDGsBiDn8GTjV1fwBS3wTk2Cx0on0Xm24B9XQATn4lZWunkTwDWUSrD5nu8ZgDGV3SPlbU8A
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7075054854/timeline
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7072826124
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7072826124/timeline
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7071975428
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7071975428/timeline
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7071690356
Making call: https://europe.api.riotgames.com/lol/match/v5/matches/EUW1_7071690356/timeline
Making ca

In [68]:
data = pd.read_csv("./match_info_csvs/kitkat_miffy_Kai'sa_early_diff.csv")
#data = pd.read_csv("./pro_match_info_csvs/doitanyway_kr1_Kai'Sa_early_diff.csv")
#print(data)

# Separate the data into wins and losses
wins = data[data['win'] == True]
losses = data[data['win'] == False]

print(wins)
print(losses)

    Unnamed: 0    match_id  cs_diff_5  cs_diff_10  cs_diff_15  gold_diff_5  \
1            1  5085624841          3          -9       -20.0         -699   
2            2  5084744149          2           1        -1.0          100   
4            4  5083862794         -5          21         4.0           74   
5            5  5083838718          0           5        21.0          312   
6            6  5083835160          2          13         NaN          632   
7            7  5083804706          8         -13       -43.0         -121   
9            9  5082973485          9          29        51.0          461   
10          10  5082937712          7          19        28.0          177   
13          13  5061267750         10          21        33.0          612   
17          17  5059217740          6           4        18.0          427   
18          18  5057244326          7          13        26.0          126   
19          19  5057189634          5          25        46.0   

In [31]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

pd.set_option('display.max_rows', 100) 

df_normal = pd.read_csv("./match_info_csvs/kitkat_miffy_Kai'Sa_match_info.csv")

df_normal = df_normal.drop(df_normal.columns[0], axis=1).drop(columns=['match_id', 'nexus_lost'])

features_normal = df_normal.drop('win', axis=1)
target_normal = df_normal['win']

X_train_normal, X_test_normal, y_train_normal, y_test_normal = train_test_split(features_normal, target_normal, test_size=0.2, random_state=42)

rf_normal = RandomForestClassifier(n_estimators=500, random_state=301)
rf_normal.fit(X_train_normal, y_train_normal)
y_pred_normal = rf_normal.predict(X_test_normal)

param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=30), param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train_normal, y_train_normal)

rf_best = grid_search.best_estimator_

y_pred_normal = rf_best.predict(X_test_normal)
print(classification_report(y_test_normal, y_pred_normal))

importances_normal = rf_best.feature_importances_
feature_names_normal = features_normal.columns

importance_df_normal = pd.DataFrame({
    'Feature': feature_names_normal,
    'Importance': importances_normal
})

importance_df_normal = importance_df_normal.sort_values(by='Importance', ascending=False)
print(importance_df_normal)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.1s
[CV] END ma

In [52]:
from location import gather_kill_data_master
import cassiopeia as cass

matches = [5102806545]

a_summoner = cass.get_summoner(puuid='toFf7CjlPKbrKT_wVp_qulclDn5ZX1vR60vSAfM-qvIlTDtpaGGAQTCkm1u5MmSxCt6akQutwKuRnA', region = "NA")

#df = gather_kill_data_master(matches, a_summoner)
#df.to_csv("ex.csv")

In [71]:
from cassiopeia import Side
def tower_location(x, y, side):
    map_size = 14500
    if y > -x + map_size:
        return 'Own' if side == 'Blue' else 'Opponent'
    else:
        return 'Opponent' if side == 'Blue' else 'Own'

from cassiopeia import Tower
import math
match = cass.get_match(id=5102806545, region="NA")
timeline = match.timeline
for participant in match.participants:
 
    if participant.summoner == a_summoner:
        user_participant = participant
        user_side = 'Blue' if user_participant.side == Side.blue else 'Red'
        print(participant.stats.win)

'''
for frame in timeline.frames:
    for event in frame.events:
        if event.type == "BUILDING_KILL" and event.building_type == "TOWER_BUILDING":
            timedelta_obj = pd.to_timedelta(event.timestamp)
            event_minute = math.ceil(timedelta_obj.total_seconds() / 60)
            print(event_minute)
            print(event.lane_type)
            print(tower_location(event.position.x, event.position.y, user_side))
            print(event.tower_type)
        elif event.type == "BUILDING_KILL" and event.building_type == "INHIBITOR_BUILDING":
            pass
'''

True


'\nfor frame in timeline.frames:\n    for event in frame.events:\n        if event.type == "BUILDING_KILL" and event.building_type == "TOWER_BUILDING":\n            timedelta_obj = pd.to_timedelta(event.timestamp)\n            event_minute = math.ceil(timedelta_obj.total_seconds() / 60)\n            print(event_minute)\n            print(event.lane_type)\n            print(tower_location(event.position.x, event.position.y, user_side))\n            print(event.tower_type)\n        elif event.type == "BUILDING_KILL" and event.building_type == "INHIBITOR_BUILDING":\n            pass\n'

Blue side top t1: 981, 10441
Blue side bot t1: 10504, 1029
Red side bot t1: 13866, 4505
Blue side top t2: 1512. 6699
Blue side mid t1: 